In [86]:
import pandas as pd

# 유튜브 인기 동영상 데이터
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60399 entries, 0 to 60398
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      60399 non-null  int64 
 1   title           60399 non-null  object
 2   channelTitle    60399 non-null  object
 3   categoryId      60399 non-null  int64 
 4   view_count      60399 non-null  int64 
 5   likes           60399 non-null  int64 
 6   dislikes        60399 non-null  int64 
 7   comment_count   60399 non-null  int64 
 8   channelId       60399 non-null  object
 9   trending_date2  60399 non-null  object
dtypes: int64(6), object(4)
memory usage: 4.6+ MB


In [65]:
# Q1 인기동영상 제작횟수 -> 정렬해서 10개 출력
# channelId 카운트 -> 
# value_counts: series를 반환하므로 
ans = list(df[df['channelId'].isin(df.channelId.value_counts().head(10).index)]['channelTitle'].unique())
ans

['장삐쭈',
 '총몇명',
 '파뿌리',
 '짤툰',
 '런닝맨 - 스브스 공식 채널',
 '엠뚜루마뚜루 : MBC 공식 종합 채널',
 'SPOTV',
 '채널 십오야',
 '이과장',
 'BANGTANTV']

In [66]:
# Q2
# dislikes > like 를 만족하는 영상 필터링
# 채널 title
# print(df[df['dislikes'] > df['likes']])
ans = list(df[df['dislikes']>df['likes']].channelTitle.unique())
print(ans)

['핫도그TV', 'ASMR 애정TV', '하얀트리HayanTree', '양팡 YangPang', '철구형 (CHULTUBE)', '왜냐맨하우스', '(MUTUBE)와꾸대장봉준', '오메킴TV', '육지담', 'MapleStory_KR', 'ROAD FIGHTING CHAMPIONSHIP', '사나이 김기훈', '나혼자산다 STUDIO', 'Gen.G esports']


In [67]:
# Q3
# channelId는 동일한데, channelTitle이 다른 경우 count
ans = df[['channelId', 'channelTitle']].drop_duplicates()
res = ans.channelId.value_counts()
# 여기서 res는 series객체!! (컬럼 하나) -> 바로 조건 걸 수 o
# value_counts()의 값이 1보다 큰 것 == 타이틀의 등장횟수가 여러 번 인 것
target = res[res>1]
print(len(target))

71


In [87]:
# Q4
# 일요일에 인기있었던 영상들 중 가장 많은 categoryId
# trending_date2.weekday 가 6
df['trending_date2'] = pd.to_datetime(df['trending_date2'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60399 entries, 0 to 60398
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Unnamed: 0      60399 non-null  int64         
 1   title           60399 non-null  object        
 2   channelTitle    60399 non-null  object        
 3   categoryId      60399 non-null  int64         
 4   view_count      60399 non-null  int64         
 5   likes           60399 non-null  int64         
 6   dislikes        60399 non-null  int64         
 7   comment_count   60399 non-null  int64         
 8   channelId       60399 non-null  object        
 9   trending_date2  60399 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(6), object(3)
memory usage: 4.6+ MB


In [71]:
# idxmax 사용: 가장 큰 값을 가진 index를 반환
# !!!!! value_counts()의 반환값은 series 객체!!!
ans = df[df['trending_date2'].dt.weekday == 6].categoryId.value_counts().idxmax()
print(ans)

'''
ans = df[df['trending_date2'].dt.weekday == 6].categoryId.value_counts()
print(ans.index[0])
print(ans.index.values[0])
'''

24


"\nans = df[df['trending_date2'].dt.weekday == 6].categoryId.value_counts()\nprint(ans.index[0])\nprint(ans.index.values[0])\n"

In [88]:
# Q5
# 요일별 인기 영상들의 categoryId 개수 -> 데이터 프레임으로 표현
# .pivot
group = df.groupby([df['trending_date2'].dt.day_name(), 'categoryId'], as_index=False).size()
answer = group.pivot(index='categoryId', columns='trending_date2')
display(answer)

size                                                  
trending_date2 Friday Monday Saturday Sunday Thursday Tuesday Wednesday
categoryId                                                             
1                 243    263      255    274      246     257       234
2                 120    105      119     99      128     119       129
10                833    837      776    830      890     894       917
15                187    215      198    217      207     208       207
17                633    668      592    636      682     708       706
19                 90     92       87     91       92      89        85
20                283    298      296    289      282     285       291
22               1288   1373     1289   1337     1341    1375      1333
23                568    594      570    556      560     569       566
24               2976   3148     3066   3096     2954    3084      3090
25                444    453      422    437      470     452       468
26                369    378      364    363      375     394       385
27                183    205      183    199      194     194       212
28                171    160      173    167      166     161       165
29                 12     10       10      9       13      11        12

In [101]:
# Q6
# comment_count / view_count 가 가장 높은 영상
# view_count = 0 인 경우는 제외
target2 = df.loc[df.view_count!=0]
t = target2.copy()
t['ratio'] = (target2['comment_count']/ target2['view_count']).dropna()
result = t.sort_values(by='ratio', ascending=False).head(1).title
print(result.values[0])
# print(result.head(1).title.values[0])

60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)


In [108]:
# Q7
# comment_count / view_count 가 최소인 영상(view_count, ratio= 0인 경우는 제외)
ratio = (df['comment_count'] / df['view_count']).dropna().sort_values()
ratio[ratio!=0].index[0]

result = df.iloc[ratio[ratio!=0].index[0]]
print(result.title)


Join the BTS #PermissiontoDance Challenge only on YouTube #Shorts


In [112]:
# Q8
# dislike / like 가 적은 영상 
# 단, like, dislike == 0 인 건 제외

'''
target = df.loc[(df.likes != 0 ) & (df.dislikes!=0)]
num = (target['dislikes']/target['likes']).sort_values().index[0] -> 인덱스!!

answer = df.iloc[num].title
print(answer)
'''

ratio = (df['dislikes']/df['likes']).dropna().sort_values()
res = df.iloc[ratio[ratio!=0].index[0]]
print(res.title)

[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰


In [125]:
# 9
# 가장 많은 트렌드 영상 제작 채널
# 채널 id 기준 빈도수 계산

ans = df[df['channelId'] == df['channelId'].value_counts().index[0]].channelTitle.unique()[0]
print(ans)

짤툰


In [137]:
# 10
ans = df[['title','channelId']].value_counts()
# ans = df.groupby(['title', 'channelId']).value_counts() # xxx -> groupby -> value_counts 존재 x
print(len(ans[ans>=20]))

40


### 유튜브 공범컨텐츠 동영상 데이터

In [139]:
import pandas as pd
channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')
display(channel.head())
display(video.head())

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


,channelid,subcnt,viewcnt,videocnt,ct,channelname
0,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,410238653,736,2021-09-30 03:01:03,꽈뚜룹
1,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,736,2021-09-30 09:01:03,꽈뚜룹
2,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,735,2021-09-30 15:01:03,꽈뚜룹
3,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,737,2021-09-30 21:01:03,꽈뚜룹
4,UCkQCwnkQfgSuPTTnw_Y7v7w,1320000,412531322,737,2021-10-01 03:01:04,꽈뚜룹


,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
0,c5JQp6xafqc,1667010,30474,706,0,6587,2021-10-10 15:20:03,공범 EP1
1,c5JQp6xafqc,1669089,30495,707,0,6589,2021-10-10 15:30:03,공범 EP1
2,c5JQp6xafqc,1674759,30522,711,0,6596,2021-10-10 15:40:02,공범 EP1
3,c5JQp6xafqc,1677026,30555,712,0,6604,2021-10-10 15:50:03,공범 EP1
4,c5JQp6xafqc,1681824,30585,713,0,6600,2021-10-10 16:00:03,공범 EP1
